## Importing Data and Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import scipy.stats as stats
from sklearn.feature_selection import mutual_info_classif
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/diabetes_25k_for_feature_selection.csv')
df.head()

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,glyburide-metformin,diabetesMed,readmitted,preceding_year_visits,number_changes,insulin_treatment
0,Caucasian,1,85,Transferred to another medical facility,Transferred from another health care facility,4,56,0,4,Genitourinary,Infectious and Parasitic,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,1,0,1,0,other_meds
1,Caucasian,1,75,Discharged to home,Referral,1,46,3,13,Circulatory,Circulatory,Circulatory,8,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,no_med
2,AfricanAmerican,1,55,Not Available,Emergency,5,52,1,23,Musculoskeletal System and Connective Tissue,Infectious and Parasitic,Mental Disorders,5,None,None,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,1,0,2,0,insulin_only
3,Caucasian,1,65,Discharged to home,Referral,5,27,2,28,Neoplasms,Neoplasms,Respiratory,8,None,None,-2,-2,-2,-2,-2,1,-2,-2,0,-2,1,0,0,1,insulin_combo
4,Caucasian,0,85,Transferred to another medical facility,Referral,11,73,0,23,Circulatory,Circulatory,Circulatory,9,None,>8,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,1,0,3,1,insulin_only


In [ ]:
df.shape

(25000, 30)

## Preparing Data

In [ ]:
X = df.drop('readmitted', 1)
X_dum = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
X.shape, X_dum.shape, y.shape

((25000, 29), (25000, 97), (25000,))

In [ ]:
X_dum.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_dum.columns]

In [ ]:
models = []

models.append(('LGB',LGBMClassifier(random_state = 0, n_jobs = -1)))
models.append(('LR', Pipeline([("Transformer", PowerTransformer()),
                               ("Scaler", StandardScaler()),
                               ("LogReg", LogisticRegression(random_state = 0, solver = 'liblinear', n_jobs = -1))])))
models.append(('KNN', Pipeline([("Transformer", PowerTransformer()),
                               ("Scaler", StandardScaler()),
                               ("KNN", KNeighborsClassifier(n_jobs = -1))])))
models.append(('DT', DecisionTreeClassifier(random_state = 0)))
models.append(('BC',BaggingClassifier(random_state = 0, n_jobs = -1)))
models.append(('ET',ExtraTreesClassifier(random_state = 0, n_jobs = -1)))
models.append(('RF', RandomForestClassifier(random_state = 0, n_jobs = -1)))
models.append(('ADA',AdaBoostClassifier(random_state = 0)))
models.append(('GB',GradientBoostingClassifier(random_state = 0)))
models.append(('XGB', XGBClassifier(random_state = 0, n_jobs = -1)))

## Sequential Forward Selection

In [ ]:
X_dum.shape

(25000, 97)

In [ ]:
model = XGBClassifier(random_state = 0, n_jobs = -1)

# Create an SFS object
sfs = SFS(estimator = model,
          k_features = (1, 97),
          forward = True,       # Set forward to True when we want to perform SFS
          scoring = 'recall',
          cv = 3)               # The number of cross-validations to perform is 3

# Train SFS with our dataset
sfs = sfs.fit(X_dum, y)

In [ ]:
# Show the performance of each subset of features considered by SFS
sfs_results = pd.DataFrame.from_dict(sfs.subsets_).T.sort_values(by = 'avg_score', ascending = False) 
sfs_results

,feature_idx,cv_scores,avg_score,feature_names
21,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.010224948875255624, 0.0040941658137154556, ...",0.00647894,"(gender, number_diagnoses, rosiglitazone, diab..."
22,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.010224948875255624, 0.0040941658137154556, ...",0.00647894,"(gender, number_diagnoses, rosiglitazone, diab..."
23,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.010224948875255624, 0.0040941658137154556, ...",0.00647894,"(gender, number_diagnoses, rosiglitazone, diab..."
19,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.009202453987730062, 0.0040941658137154556, ...",0.00613811,"(gender, number_diagnoses, rosiglitazone, diab..."
18,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 47, 49,...","[0.009202453987730062, 0.0040941658137154556, ...",0.00613811,"(gender, number_diagnoses, rosiglitazone, diab..."
20,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.009202453987730062, 0.0040941658137154556, ...",0.00613811,"(gender, number_diagnoses, rosiglitazone, diab..."
25,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.010224948875255624, 0.0030706243602865915, ...",0.00613776,"(gender, number_diagnoses, rosiglitazone, diab..."
24,"(0, 6, 14, 17, 18, 19, 26, 32, 34, 40, 44, 47,...","[0.010224948875255624, 0.0030706243602865915, ...",0.00613776,"(gender, number_diagnoses, rosiglitazone, diab..."
15,"(0, 6, 14, 17, 18, 19, 34, 40, 47, 49, 51, 72,...","[0.0081799591002045, 0.0040941658137154556, 0....",0.00579728,"(gender, number_diagnoses, rosiglitazone, diab..."
16,"(0, 6, 14, 17, 18, 19, 32, 34, 40, 47, 49, 51,...","[0.0081799591002045, 0.0040941658137154556, 0....",0.00579728,"(gender, number_diagnoses, rosiglitazone, diab..."


In [ ]:
sfs_results.to_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/25K_XGBoost_Forward_Results_Saurabh.csv', index = False)

In [ ]:
sfs_results.iloc[0, 3]

('gender',
 'number_diagnoses',
 'rosiglitazone',
 'diabetesMed',
 'preceding_year_visits',
 'number_changes',
 'discharge_disposition_id_Not_Available',
 'diag_1_Circulatory',
 'diag_1_Diabetes',
 'diag_1_Injury_and_Poisoning',
 'diag_1_Nervous',
 'diag_1_Respiratory',
 'diag_1_Skin_and_Subcutaneous_Tissue',
 'diag_2_Congenital_Anomalies',
 'diag_2_Diabetes',
 'diag_2_Genitourinary',
 'diag_3_Digestive',
 'diag_3_Neoplasms',
 'diag_3_Not_Required',
 'max_glu_serum__300',
 'A1Cresult__8')

## Sequential Backward Selector

In [ ]:
model = XGBClassifier(random_state = 0, n_jobs = -1)

# Create an SFS object
sbs = SFS(estimator = model,
          k_features = (1, 97),
          forward = False,       # Set forward to True when we want to perform SFS
          scoring = 'recall',
          cv = 3)               # The number of cross-validations to perform is 3

# Train SFS with our dataset
sbs = sbs.fit(X_dum, y)

TypeError: ignored

In [ ]:
# Show the performance of each subset of features considered by SFS
sbs_results = pd.DataFrame.from_dict(sbs.subsets_).T.sort_values(by = 'avg_score', ascending = False) 
sbs_results

In [ ]:
sbs_results.to_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/25K_XGBoost_Backward_Results_Saurabh.csv', index = False)